In [2]:
import os
from tqdm import tqdm_notebook
from utils import *
from calc_consistency import *
from utils import BlurPooling1D,excute_command
from keras.utils import CustomObjectScope

Using TensorFlow backend.
/Users/hu/.pyenv/versions/3.6.6/envs/Biosignals/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/hu/.pyenv/versions/3.6.6/envs/Biosignals/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
new_file_name = "new_result.csv"

## Pool1

In [3]:
result_roots = ["./Result_CNN_pool1/results_max/",
               "./Result_CNN_pool1/results_maxblur-/",
               "./Result_CNN_pool1/results_maxblur-1/",
               "./Result_CNN_pool1/results_maxblur-2/",
               "./Result_CNN_pool1/results_maxblur-3/",
               "./Result_CNN_pool1/results_maxblur-4/",
               "./Result_CNN_pool1/results_maxblur-5/",
               "./Result_CNN_pool1/results_maxblur-6/",
               "./Result_CNN_pool1/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])

#new_result_file = os.path.join(result_root,"new_result.csv")
#result_df = pd.read_csv(new_result_file)
#result_df = result_df.drop("_",axis=1)
#result_df

## Pool2

In [4]:
result_roots = ["./Result_CNN_pool2/results_max/",
               "./Result_CNN_pool2/results_maxblur-/",
               "./Result_CNN_pool2/results_maxblur-1/",
               "./Result_CNN_pool2/results_maxblur-2/",
               "./Result_CNN_pool2/results_maxblur-3/",
               "./Result_CNN_pool2/results_maxblur-4/",
               "./Result_CNN_pool2/results_maxblur-5/",
               "./Result_CNN_pool2/results_maxblur-6/",
               "./Result_CNN_pool2/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])

#new_result_file = os.path.join(result_root,"new_result.csv")
#result_df = pd.read_csv(new_result_file)
#result_df = result_df.drop("_",axis=1)
#result_df

## Pool3 

In [5]:
result_roots = ["./Result_CNN_pool3/results_max/",
               "./Result_CNN_pool3/results_maxblur-/",
               "./Result_CNN_pool3/results_maxblur-1/",
               "./Result_CNN_pool3/results_maxblur-2/",
               "./Result_CNN_pool3/results_maxblur-3/",
               "./Result_CNN_pool3/results_maxblur-4/",
               "./Result_CNN_pool3/results_maxblur-5/",
               "./Result_CNN_pool3/results_maxblur-6/",
               "./Result_CNN_pool3/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])

#new_result_file = os.path.join(result_root,"new_result.csv")
#result_df = pd.read_csv(new_result_file)
#result_df = result_df.drop("_",axis=1)
#result_df

## For augment

In [6]:
result_roots = ["./Result_CNN_aug_pool1/results_max/",
               "./Result_CNN_aug_pool1/results_maxblur-/",
               "./Result_CNN_aug_pool1/results_maxblur-3/",
               "./Result_CNN_aug_pool1/results_maxblur-4/",
               "./Result_CNN_aug_pool1/results_maxblur-5/",
               "./Result_CNN_aug_pool1/results_maxblur-6/",
               "./Result_CNN_aug_pool1/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])


In [7]:
result_roots = ["./Result_CNN_aug_pool2/results_max/",
               "./Result_CNN_aug_pool2/results_maxblur-/",
               "./Result_CNN_aug_pool2/results_maxblur-3/",
               "./Result_CNN_aug_pool2/results_maxblur-4/",
               "./Result_CNN_aug_pool2/results_maxblur-5/",
               "./Result_CNN_aug_pool2/results_maxblur-6/",
               "./Result_CNN_aug_pool2/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])


In [8]:
result_roots = ["./Result_CNN_aug_pool3/results_max/",
               "./Result_CNN_aug_pool3/results_maxblur-/",
               "./Result_CNN_aug_pool3/results_maxblur-3/",
               "./Result_CNN_aug_pool3/results_maxblur-4/",
               "./Result_CNN_aug_pool3/results_maxblur-5/",
               "./Result_CNN_aug_pool3/results_maxblur-6/",
               "./Result_CNN_aug_pool3/results_maxblur-7/"]

for result_root in tqdm_notebook(result_roots):
    for cross_idx in range(5):
        excute_command("calc_consistency.py",[cross_idx,result_root])
